<a href="https://colab.research.google.com/github/NishantRajora/Machine-Learning/blob/main/l_16_2_Item_besed_recomendatin_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie_ratings = pd.read_csv('ratings.csv')

In [3]:
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies = pd.read_csv('movies.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movie_data = movie_ratings.merge(movies, on ='movieId')
movie_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
user_movie_matrix = movie_data.pivot_table(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

movie_similarity = cosine_similarity(user_movie_matrix.T)

In [9]:
movie_similarity_df = pd.DataFrame(
    movie_similarity,
    index=user_movie_matrix.columns,
    columns=user_movie_matrix.columns
)

movie_similarity_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.410562,0.296917,0.035573,0.308762,0.376316,0.277491,0.131629,0.232586,0.395573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.410562,1.000000,0.282438,0.106415,0.287795,0.297009,0.228576,0.172498,0.044835,0.417693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.296917,0.282438,1.000000,0.092406,0.417802,0.284257,0.402831,0.313434,0.304840,0.242954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.035573,0.106415,0.092406,1.000000,0.188376,0.089685,0.275035,0.158022,0.000000,0.095598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.308762,0.287795,0.417802,0.188376,1.000000,0.298969,0.474002,0.283523,0.335058,0.218061,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def item_based_recommend(user_id, user_movie_matrix, similarity_df, movies, top_n=5):

    user_ratings = user_movie_matrix.loc[user_id]


    # Compute scores
    scores = similarity_df.dot(user_ratings)
    scores = scores.sort_values(ascending=False)

    # Remove movies already rated
    scores = scores[user_ratings == 0]

    recommended_movie_ids = scores.head(top_n).index

    return movies[movies['movieId'].isin(recommended_movie_ids)]

In [11]:
item_based_recommend(
    user_id=1,
    user_movie_matrix=user_movie_matrix,
    similarity_df=movie_similarity_df,
    movies=movies,
    top_n=5
)

,movieId,title,genres
793,1036,Die Hard (1988),Action|Crime|Thriller
902,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
1158,1527,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi
1445,1968,"Breakfast Club, The (1985)",Comedy|Drama
2195,2918,Ferris Bueller's Day Off (1986),Comedy


In [12]:
item_based_recommend(
    user_id=2,
    user_movie_matrix=user_movie_matrix,
    similarity_df=movie_similarity_df,
    movies=movies,
    top_n=10
)

,movieId,title,genres
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller
4935,7438,Kill Bill: Vol. 2 (2004),Action|Drama|Thriller
5917,33794,Batman Begins (2005),Action|Crime|IMAX
6331,48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
6743,59315,Iron Man (2008),Action|Adventure|Sci-Fi
7043,69122,"Hangover, The (2009)",Comedy|Crime
7212,72998,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
7214,73017,Sherlock Holmes (2009),Action|Crime|Mystery|Thriller
7395,79702,Scott Pilgrim vs. the World (2010),Action|Comedy|Fantasy|Musical|Romance
7413,80463,"Social Network, The (2010)",Drama
